In [70]:
import pandas as pd
from openai import OpenAI

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import json

In [71]:
df = pd.read_csv("../data/Fine-tuning Data_raw.csv").drop(columns="텍스트 - ERD 생성 데이터")
df.head()

,인덱스,텍스트 - 요약/정리 데이터
0,1,"GQ 서비스는 학습 중심의 스터디 그룹 운영을 지원하는 웹 플랫폼으로, 단순한 그룹..."
1,2,‘기억박물관’은 치매 환자와 그 가족을 위한 따뜻한 기억 회복 웹사이트입니다. 사용...
2,3,"이 앱은 보행 약자들을 위한 지하철 길찾기 앱으로, 계단 없이 이동할 수 있는 경로..."
3,4,"이 앱은 1인 가구를 위한 맞춤형 플랫폼으로, 주로 커뮤니티 기능에 중점을 두고 있..."
4,5,"이 웹앱은 응급실 및 병원 찾기 기능을 제공하여, 응급 상황에서 사용자가 빠르고 안..."


In [83]:
dataset = df.copy().drop(columns= '인덱스')
dataset.columns = ['text']
dataset

,text
0,"GQ 서비스는 학습 중심의 스터디 그룹 운영을 지원하는 웹 플랫폼으로, 단순한 그룹..."
1,‘기억박물관’은 치매 환자와 그 가족을 위한 따뜻한 기억 회복 웹사이트입니다. 사용...
2,"이 앱은 보행 약자들을 위한 지하철 길찾기 앱으로, 계단 없이 이동할 수 있는 경로..."
3,"이 앱은 1인 가구를 위한 맞춤형 플랫폼으로, 주로 커뮤니티 기능에 중점을 두고 있..."
4,"이 웹앱은 응급실 및 병원 찾기 기능을 제공하여, 응급 상황에서 사용자가 빠르고 안..."
...,...
95,"지역 축제의 혼잡도를 낮추고 운영 효율을 높이기 위해, 관람객의 이동 경로와 체류 ..."
96,작물의 생장 조건을 분석하고 환경을 자동으로 조절하는 스마트팜 제어 시스템을 기획하...
97,청소년의 감정 상태를 분석하고 적절한 상담 자원을 연결해주는 정서 관리 플랫폼을 기...
98,가정에서 발생하는 음식물 쓰레기를 줄이기 위한 스마트 분리 수거 시스템을 기획하였다...


In [72]:
with open('../data/my_api_key.txt', 'r') as file:
    API_KEY = file.read().strip()

In [90]:
def ERD_data(df):

  entities_lst = []
  relationships_lst = []

  sys_template = """
  당신은 사용자가 입력한 문장의 개요를 예상 ERD를 작성해주는 프로그램입니다. 
  주제에 대해서 예상되는 ERD를 json 형식으로 답변해주세요.

  중요: 반드시 유효한 JSON 형식으로만 응답하세요. 다음 형식을 정확히 따르세요:

  {{
    "entities": [
      {{
        "name": "User",
        "attributes": [
          {{
            "name": "user_id",
            "data_type": "INTEGER",
            "is_primary_key": true,
            "is_foreign_key": false,
            "is_nullable": false
          }},
          {{
            "name": "username",
            "data_type": "VARCHAR(50)",
            "is_primary_key": false,
            "is_foreign_key": false,
            "is_nullable": false
          }},
          {{
            "name": "email",
            "data_type": "VARCHAR(100)",
            "is_primary_key": false,
            "is_foreign_key": false,
            "is_nullable": true
          }}
        ]
      }}
    ],
    "relationships": [
      {{
        "from": "User",
        "to": "Order",
        "type": "one-to-many",
        "foreign_key": "user_id",
        "constraint_name": "fk_order_user"
      }}
    ]
  }}

  각 속성의 상세 정보(PK, FK, NULL 허용 여부, 데이터 타입)를 모두 포함하세요.
  다른 텍스트, 설명, 마크다운은 절대 포함하지 마세요.
  """
  user_template = "{question}"
  
  prompt = ChatPromptTemplate.from_messages([
    ("system", sys_template),
    ("user", user_template)
  ])

  llm = ChatOpenAI(
      api_key=API_KEY,
      model='gpt-4o-mini',
      temperature=0
  )

  llm_chain = prompt | llm

  for i in range(len(df)) :
    text = df.iloc[i,1]
    answer = llm_chain.invoke({"question": text})
    data = json.loads(answer.content)
    entities_lst.append(data['entities'])
    relationships_lst.append(data['relationships'])
    
  return entities_lst, relationships_lst 

In [ ]:
entities_lst, relationships_lst = ERD_data(df)

dataset['ERD_data'] = entities_lst
dataset['relationships_data'] = relationships_lst

In [ ]:
dataset.to_csv("../data/ERD_data.csv")